# ETH and Sol domains price comparaison
On part du postulat que SOL gagne fortement et va continuer de monter en marketshare.
Les domaines SOL sont underlooked and misspricé à mon avis. Ce tool va permettre de voir les ventes sur les ENS ETH et les comparer aux SNS pour obtenir les meilleures opportunités.
## Test des APIS
Il nous faut un RPC Sol pour accéder au resolver SNS.
On utilise Alchemy.

In [137]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime
import os
# Define helpers to connect to Bitmex API.
import json
import urllib
import hashlib
import hmac
import urllib.parse
from datetime import timedelta, datetime
from dateutil.parser import parse
import time
import pickle
import numpy as np
load_dotenv()

True

## ENS
### Define ENS functions

In [61]:
def get_all_ens_sales(start_time: datetime, end_time: datetime):
    # query params
    params = {
        "startTimestamp": start_time.timestamp(),
        "endTimestamp": end_time.timestamp(),
        "limit": 30,
        "contract": "0x57f1887a8BF19b14fC0dF6Fd9B2acc9Af147eA85",
        "includeTokenMetadata":True,
        'sortBy':'price'
    }
    headers = {"x-api-key": os.getenv('RESERVOIR_API_KEY')}
    df_ens_sales_eth = pd.DataFrame()

    def loop_over_continuation(df_ens_sales_eth):
        resp = requests.get("https://api.reservoir.tools/sales/v6",params=params, headers=headers)
        resp_json = resp.json()
        df_ens_sales_eth = pd.concat([df_ens_sales_eth, pd.DataFrame(resp_json['sales'])])
        sales = resp_json['sales']
        print(sales[-1]['price']['amount']['usd'])
        if resp_json['continuation'] and sales[-1]['price']['amount']['usd']>1000:
            params['continuation'] = resp_json['continuation']
            return loop_over_continuation(df_ens_sales_eth)
        else:
            print('Finished')
            return df_ens_sales_eth
    return loop_over_continuation(df_ens_sales_eth)

def ens_cleaning(df):
    df_ens_sales = df.copy()
    df_ens_sales['price_eth']=df_ens_sales.apply(lambda x: x['price']['amount']['native'] if (x['price']['currency']['contract']=='0x0000000000000000000000000000000000000000' or x['price']['currency']['contract']=='0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2') else None, axis=1)
    df_ens_sales['domain']=df_ens_sales.apply(lambda x:str(x['token']['name'])[:-4], axis=1)
    df_ens_sales['price_usd']=df_ens_sales.apply(lambda x: x['price']['amount']['usd'],axis=1)
    # Get average of sales for each domain
    df_ens_avg_price = df_ens_sales.groupby('domain')['price_usd'].mean().reset_index()
    df_ens_avg_price = df_ens_avg_price.rename(columns={'price_usd': 'avg_price_usd'})
    df_ens_count_sales = df_ens_sales.groupby('domain').count().reset_index()
    # Rename the count column to 'nb_sales' for clarity
    df_ens_count_sales = df_ens_count_sales.rename(columns={'price_eth': 'nb_sales'})
    df_ens_sales = df_ens_sales.merge(df_ens_count_sales[['domain', 'nb_sales']], on='domain', how='left')
    df_ens_sales = df_ens_sales.merge(df_ens_avg_price, on='domain', how='left')
    df_ens_sales.drop_duplicates(subset=['domain'], inplace=True)
    df_ens_sales=df_ens_sales[['domain', 'avg_price_usd', 'nb_sales','timestamp','txHash','fillSource','orderKind']]
    return df_ens_sales

### Fetch ENS data

In [26]:
df_ens_sales_raw=get_all_ens_sales(datetime(2022, 5, 1), datetime(2022, 5, 20))
df_ens_sales_raw

10396.53766
5339.43956
5147.76738
4901.3317
4764.423
4600.13255
4296.16206
4107.2612
4068.92677
3978.56702
3833.44379
3657.37919
3482.9575
3285.80896
3099.23887
2738.17414
2710.79239
2669.71978
2628.64717
2601.26543
2595.47168
2464.35672
2354.82976
1642.90448
794.0705
Finished


,id,saleId,token,orderId,orderSource,orderSide,orderKind,from,to,amount,...,batchIndex,timestamp,price,washTradingScore,paidFullRoyalty,feeBreakdown,isDeleted,createdAt,updatedAt,marketplaceFeeBps
0,fb07f7d7ff8cb91b024fd83999c252c0f3676f9a021596...,c0431f156705663a83a8fcefae58be1c706479c009e7d4...,{'contract': '0x57f1887a8bf19b14fc0df6fd9b2acc...,0xb978cb877472bea4013d66c4a8b36993cbb90379ec07...,opensea.io,ask,wyvern-v2.3,0xcf815caa0ce4dfba248ff328b9b5282a87e26a89,0x6d04403de6dd8952d683fb02f5fba591419c9556,1,...,1,1651427327,{'currency': {'contract': '0x00000000000000000...,0,True,[],False,2022-05-01T17:48:47.000Z,2023-03-10T21:05:12.077Z,NaN
1,72df4193e972bd7af10cf8302e6b2d23aff45400970bc9...,8fe7cbdf9e80c0580df7c35d62a8b90432c43803428dd1...,{'contract': '0x57f1887a8bf19b14fc0df6fd9b2acc...,0x462bfe2a1cf653dd7f83307e9004a511d2ef12d60aae...,opensea.io,bid,wyvern-v2.3,0x986b3b04523c0fd690b2fcf7cd114fc2b7d9e740,0xcfe7321fd5aca6912694ecf85d41617e0d884fdc,1,...,1,1651364907,{'currency': {'contract': '0xc02aaa39b223fe8d0...,0,True,[],False,2022-05-01T00:28:27.000Z,2023-03-10T22:52:48.176Z,NaN
2,99df7c5b18564f0c2b4238e6e9abb12550c5cff15b06f1...,65a685ed191453f5be252f6cccdb7e9f1ddab4dcbfd7e1...,{'contract': '0x57f1887a8bf19b14fc0df6fd9b2acc...,0x8201db6b22109d992bdff7a21bfb1980caa844b9492d...,opensea.io,ask,wyvern-v2.3,0xaa8cdc25e3d1a4c4ee007595a9ca6b77b87f4600,0xdaa3a96173169a2efbf716092d7b8598b9dba27c,1,...,1,1651425356,{'currency': {'contract': '0x00000000000000000...,0,True,[],False,2022-05-01T17:15:56.000Z,2023-03-10T21:09:05.633Z,NaN
3,a8ffb181b8167a1d8fc057183005b016141b515970fcf4...,7a8eaf1d6e1aeb6dcdac567224b225d9f911b3fe77ac31...,{'contract': '0x57f1887a8bf19b14fc0df6fd9b2acc...,0x89ef9e74cf951c045ed0983ef8f08ae3b285ab23b0ac...,opensea.io,ask,wyvern-v2.3,0x0119f93047266e21c92aed02e42efbf9cdb5b0cf,0x1ce0750d7177a3a19eba48c32d5c86912fbb5e69,1,...,1,1651412564,{'currency': {'contract': '0x00000000000000000...,0,True,[],False,2022-05-01T13:42:44.000Z,2023-03-10T21:37:13.798Z,NaN
4,2a8f0718945dd8adcf6a48273ed6752352ada53a67066d...,4ea22c194f1d00847246ac9457bb1a556d99acba084d8d...,{'contract': '0x57f1887a8bf19b14fc0df6fd9b2acc...,0x1b8129df72c6d177d59e233217f084ed03afede1ec12...,looksrare.org,ask,looks-rare,0xfb30cdae07272c8a37929039200201926eba6428,0x9609e6d6cbd911befc7b883816822ba355ec9eaf,1,...,1,1651427610,{'currency': {'contract': '0x00000000000000000...,0,True,[],False,2022-05-01T17:53:30.000Z,2023-03-10T21:04:50.768Z,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,13e6b3077a6e74f535b412ca4037cf044ba8667c631b59...,683ac89a6d6625de20526830e9b39e1156c58f67d2796f...,{'contract': '0x57f1887a8bf19b14fc0df6fd9b2acc...,0xc2d98370e593a8067a0c3570327c6980f7efeca6dac7...,opensea.io,bid,wyvern-v2.3,0x6b58007b960016b2f559dbfd809ac4dcb1febdfe,0x328ebc7bb2ca4bf4216863042a960e3c64ed4c10,1,...,1,1651399075,{'currency': {'contract': '0xc02aaa39b223fe8d0...,0,True,[],False,2022-05-01T09:57:55.000Z,2023-03-10T21:57:43.967Z,NaN
26,f6216c05116a272c89bc1d383015c45398b4d35b74db1b...,6a483c26c4ce43301c48f8c71c3a4781727e550e9e62a3...,{'contract': '0x57f1887a8bf19b14fc0df6fd9b2acc...,0x8bf46f10fc79f43ee2fa915cc657cbc4f3c725e83636...,opensea.io,ask,wyvern-v2.3,0x64136535ee6472ebabb6531d4edfc2626f2146e1,0x0000000035634b55f3d99b071b5a354f48e10bef,1,...,1,1651425711,{'currency': {'contract': '0x00000000000000000...,0,True,[],False,2022-05-01T17:21:51.000Z,2023-03-10T21:08:42.548Z,NaN
27,49b89841bd4d35010121a33da9472006859e45b06640cb...,04b5129bea619065cdaa148650e0f618c1531d99d9fcab...,{'contract': '0x57f1887a8bf19b14fc0df6fd9b2acc...,0x94ea05cd7aa6a8feb5ffbf708e76a3395d382a072d62...,opensea.io,ask,wyvern-v2.3,0xd711e8975f248216dba4730c06fc86b9aa765f92,0xe41a26bf902930b5ef6998bb4e53dc0ad7fee599,1,...,1,1651423233,{'currency': {'contract': '0x00000000000000000...,0,True,[],False,2022-05-01T16:40:33.000Z,2023-03-10T21:12:10.162Z,NaN
28,884d11a5b48a44f35c2caa452aab7f2063d6bbb49a8353...,76a6b20c56e61ebacdc355a3b

In [35]:
df = df_ens_sales_raw.copy()
df['price_eth']=df.apply(lambda x: x['price']['amount']['native'] if (x['price']['currency']['contract']=='0x0000000000000000000000000000000000000000' or x['price']['currency']['contract']=='0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2') else None, axis=1)
df['domain']=df.apply(lambda x:str(x['token']['name'])[:-4], axis=1)
df['price_usd']=df.apply(lambda x: x['price']['amount']['usd'],axis=1)
# Get average of sales for each domain
df_ens_avg_price = df.groupby('domain')['price_usd'].mean().reset_index()
df_ens_avg_price

,domain,price_usd
0,,3326.881575
1,0013,10396.537660
2,0053,5634.979760
3,0069,5476.348270
4,0126,3450.099410
...,...,...
691,x4n4x,821.452240
692,yeezy,4131.269050
693,美高梅.et,1039.196130
694,📬📬📬.et,1369.087070


## Clean ENS df and doing computations

In [39]:
ens_cleaning(df_ens_sales_raw)

,domain,avg_price_usd,nb_sales,timestamp,txHash,fillSource,orderKind
0,085,41620.24687,1,1651427327,0x20a76c664cc70c8b36e5bbf84f179a717ce58da4de13...,opensea.io,wyvern-v2.3
1,114,41072.61204,1,1651364907,0xab8a69806251c2fddb24e4cc2a23fd47d10adf72105f...,opensea.io,wyvern-v2.3
2,027,41072.61204,1,1651425356,0x11d1bc0001d6f65c4492a8732177962ac24966fe6c04...,opensea.io,wyvern-v2.3
3,135,36965.35084,1,1651412564,0x19d2fa04df984354a6e7b0ce05d32620a7189230b5c5...,opensea.io,wyvern-v2.3
4,179,35596.26377,1,1651427610,0xe2675d63e80ba84c924dda119326d7ddf38788d50082...,looksrare.org,looks-rare
...,...,...,...,...,...,...,...
745,x4n4x,821.45224,1,1651399075,0x6e2f0db7f66d1ae3e88ffbde5e7434fb28e9567bb7c0...,opensea.io,wyvern-v2.3
746,rapidash,821.45224,1,1651425711,0x70167e9a6e6d0d5899223fada1c0c49c80e583913036...,gem.xyz,wyvern-v2.3
747,767676,821.45224,1,1651423233,0xdddbaa5e6b3ea575dc183bd3bc6486116a0823403bce...,opensea.io,wyvern-v2.3
748,06969,821.45224,1,1651422857,0xb24ef5e26fbcfd6730802ba8286a127d241c59d82586...,opensea.io,wyvern-v2.3


## SNS
### Define SNS functions

In [62]:
# Get all SNS listings
def get_all_sns_listings():
    limit = 20
    url = "https://api-mainnet.magiceden.dev/v2/collections/bonfida/listings"
    params = {
        "limit": limit,  # Unknown limit, to be tested
        "offset": 0,
    }
    df_sns_listings = pd.DataFrame()

    def loop_over_continuation(df_sns_listings):
        start_loop = time.time()
        resp = requests.get(url, params=params)
        resp_json = resp.json()
        df_sns_listings_iteration = pd.DataFrame(resp_json)
        df_sns_listings = pd.concat([df_sns_listings, df_sns_listings_iteration])

        if not df_sns_listings_iteration.empty:
            params["offset"] += limit
            print(resp_json[-1]['price'])
            time_taken = time.time() - start_loop
            time.sleep(0.5 - time_taken if time_taken < 0.5 else 0)
            return loop_over_continuation(df_sns_listings)
        else:
            return df_sns_listings

    return loop_over_continuation(df_sns_listings)


def sns_cleaning(df):
    df_sns_listings = df.copy()
    df_sns_listings['price_sol'] = df_sns_listings['price']
    df_sns_listings['domain'] = df_sns_listings.apply(lambda x: x['token']['name'], axis=1)
    df_bitmex_hourly_sol_prices_raw = get_sol_prices()
    df_sns_listings['price_usd'] = df_sns_listings['price_sol'] * df_bitmex_hourly_sol_prices_raw.iloc[-1]['close']
    df_sns_listings = df_sns_listings[['domain', 'price_sol', 'price_usd']]
    return df_sns_listings


def check_if_registered(domain_name: str):
    # This is using QuickNode API and SNS add-on, if SNS is bought, it automatically resolves the domain to owner if bought.
    # Therefore if result = None it means domain is available.
    try:
        resp = requests.post(
            'https://late-small-voice.solana-mainnet.quiknode.pro/213109288e418132157c5d7b3da4f0fc4af3158f/', json={
                "jsonrpc": "2.0",
                "method": "sns_resolveDomain",
                "params": [
                    domain_name,
                ],
                "id": 42
            }
            )
        result = resp.json()['result']
        print('domain', domain_name, 'result', result)
        return result != None
    except Exception as e:
        print('Exception', e)
        return None


def process_domain(domain):
    return domain, check_if_registered(domain)


# Get SOL historical price from mex
api_key_id = os.getenv('BITMEX_API_ID')
api_key_secret = os.getenv('BITMEX_API_SECRET')

# Simple GET
#
verb = 'GET'
# Note url-encoding on querystring - this is '/api/v1/instrument?filter={"symbol": "XBTM15"}'
# Be sure to HMAC *exactly* what is sent on the wire
path = 'https://www.bitmex.com/api/v1/instrument'
expires = 1518064237  # 2018-02-08T04:30:37Z
data = json.dumps({"filter": {"symbol": "XBTM15"}})
print(data)


def generate_signature(secret, verb, url, expires, data):
    """Generate a request signature compatible with BitMEX."""
    # Parse the url so we can remove the base and extract just the path.
    print(data)
    parsedURL = urllib.parse.urlparse(url)
    path = parsedURL.path
    if parsedURL.query:
        path = path + '?' + parsedURL.query

    if isinstance(data, (bytes, bytearray)):
        data = data.decode('utf8')

    message = bytes(verb + path + str(expires) + data, 'utf-8')

    signature = hmac.new(bytes(secret, 'utf-8'), message, digestmod=hashlib.sha256).hexdigest()
    return signature


def generate_headers(url):
    headers = {}
    sig_expires = int((datetime.now() + timedelta(minutes=1)).timestamp())
    signature = generate_signature(os.getenv('BITMEX_API_SECRET'), 'GET', url, sig_expires, '')
    headers['api-expires'] = str(sig_expires)
    headers['api-signature'] = signature
    headers['api-key'] = str(os.getenv('BITMEX_API_ID'))
    return headers


# Extract historical data for ETHUSD from Bitmex.
# Define the endpoint URL
def get_sol_prices():
    url = "https://www.bitmex.com/api/v1/trade/bucketed"
    count = 1000  # Max 1000
    cursor = 0
    # We can take full time length here because we will merge on timestamp from nft sales (which are filtered by from_time and to_time)
    start_time = "2018-03-02T09:02:00"
    end_time = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
    # Define the parameters
    params = {
        "symbol": "SOLUSD",
        "count": count,  # Modify this value based on your needs
        "start": cursor,  # After 11000 we dont get any result.
        "binSize": "1h",
        "columns": "close,timestamp",
    }

    df_bitmex_hourly_sol_prices_raw = pd.DataFrame()

    while True:
        encoded_params = urllib.parse.urlencode(params)
        full_url = url + "?" + encoded_params
        headers = generate_headers(full_url)

        response = requests.get(full_url, headers=headers)
        print(headers)
        print(params)
        # Check the status code
        if response.status_code == 200:
            resp_json = response.json()
            # If the least recent timestamp is > of our end time we stop the loop. It means we registered all the data we need and more.
            if len(resp_json) == 0:
                print('Finished')
                break

            # Parse the response JSON into a DataFrame.
            df_bitmex_hourly_sol_prices_raw = pd.concat([df_bitmex_hourly_sol_prices_raw, pd.DataFrame(resp_json)])
            print('First row', resp_json[0])
            # Calc the remaining API calls we can make before we hit the rate limit.
            remaining_api_calls = int(response.headers.get('x-ratelimit-remaining'))
            print(remaining_api_calls)
            # Increment the cursor by count.
            cursor += count
            # For whatever reason not detailed, cursor stops at 11000. So we need to reset the start time to get the next 11000.

            params['start'] = cursor
            print('Cursor', cursor)
            # If we dont have any remaining_api_calls we wait the given time.
            if remaining_api_calls == 0:
                time_to_sleep = int(response.headers.get('x-ratelimit-reset')) - int(time.time())
                # Time can be negative, because of the time it takes to process the data if the reset limit is very close.
                if time_to_sleep > 0:
                    print(f"Request limit reached, waiting {time_to_sleep}")
                    time.sleep(time_to_sleep)
        elif response.status_code == 429:
            must_wait = int(response.headers.get('retry-after'))
            print(f"Request limit reached, waiting {must_wait}")
            time.sleep(must_wait)
        else:
            print(f"Request failed with status code {response.status_code}")
            print(response.text)
    return df_bitmex_hourly_sol_prices_raw

### Execute and get SNS listings

In [59]:
df_sns_listings_raw = get_all_sns_listings()
df_sns_listings_raw

0.2299
0.25
0.2999
0.35


KeyboardInterrupt: 

## Clean SNS df and do computations

In [14]:
sns_cleaning(df_sns_listings_raw)


{'api-expires': '1703407773', 'api-signature': 'f2db4275c6b14f3a894f17a6d83e1ca040c0e5fd27a45edec0eb981afd33e6b9', 'api-key': 'HM_OVkhTCygs5BcVk2NY6aIi'}
{'symbol': 'SOLUSD', 'count': 1000, 'start': 0, 'binSize': '1h', 'columns': 'close,timestamp'}
First row {'symbol': 'SOLUSD', 'timestamp': '2021-10-20T04:00:00.000Z', 'close': None}
119
Cursor 1000

{'api-expires': '1703407774', 'api-signature': '30c399e427b27e8d20fb0ab77ef0a55e098bf45b0dfc3fea46941aa41fa6c616', 'api-key': 'HM_OVkhTCygs5BcVk2NY6aIi'}
{'symbol': 'SOLUSD', 'count': 1000, 'start': 1000, 'binSize': '1h', 'columns': 'close,timestamp'}
First row {'timestamp': '2021-11-30T20:00:00.000Z', 'symbol': 'SOLUSD', 'close': 215.45}
119
Cursor 2000

{'api-expires': '1703407775', 'api-signature': '6e88129c92b84a1caa9349bc8ef311149fd77bc2a1a38876de6b111fbff1b7b2', 'api-key': 'HM_OVkhTCygs5BcVk2NY6aIi'}
{'symbol': 'SOLUSD', 'count': 1000, 'start': 2000, 'binSize': '1h', 'columns': 'close,timestamp'}
First row {'timestamp': '2022-01-11T

,domain,price_sol,price_usd
0,sol1000,0.1940,22.216880
1,declubs,0.1989,22.778028
2,abuali,0.1990,22.789480
3,designersdeity,0.1990,22.789480
4,nosepass,0.2000,22.904000
5,h00ts,0.2010,23.018520
6,philipmignon,0.2090,23.934680
7,y00tsclubs,0.2100,24.049200
8,deez-y00ts,0.2100,24.049200
9,adibou,0.2100,24.049200


## SNS and ENS fetching

In [89]:
from concurrent.futures import ThreadPoolExecutor
# Run ETH fetching and SNS fetching concurrently
import threading

with ThreadPoolExecutor() as executor:
    future1 = executor.submit(get_all_ens_sales,datetime(2022, 5, 1), datetime(2023, 12, 23))
    future2 = executor.submit(get_all_sns_listings)
    df_ens_sales_raw = future1.result()  # This will wait for the thread to finish and return its result
    df_sns_listings_raw = future2.result()  # This will wait for the thread to finish and return its result

0.2299
76419.24828
0.25
0.29


KeyboardInterrupt: 

## SNS and ENS df cleaning, computation and merging

In [120]:
df_sns_listings = sns_cleaning(df_sns_listings_raw)
df_ens_sales = ens_cleaning(df_ens_sales_raw)

df_sales = df_ens_sales.merge(df_sns_listings,how='left', on='domain')
df_sales


{'api-expires': '1703442773', 'api-signature': '591f35cbc7c51f8caaf802c35c8e4a02cf756544946641d06cee5ee79d72a91a', 'api-key': 'HM_OVkhTCygs5BcVk2NY6aIi'}
{'symbol': 'SOLUSD', 'count': 1000, 'start': 0, 'binSize': '1h', 'columns': 'close,timestamp'}
First row {'timestamp': '2021-10-20T04:00:00.000Z', 'symbol': 'SOLUSD'}
119
Cursor 1000

{'api-expires': '1703442775', 'api-signature': '52ae9ce8e620fdbe5f861f313adaab789f138ed712872476b5019e76cff02b21', 'api-key': 'HM_OVkhTCygs5BcVk2NY6aIi'}
{'symbol': 'SOLUSD', 'count': 1000, 'start': 1000, 'binSize': '1h', 'columns': 'close,timestamp'}
First row {'timestamp': '2021-11-30T20:00:00.000Z', 'symbol': 'SOLUSD', 'close': 215.45}
118
Cursor 2000

{'api-expires': '1703442775', 'api-signature': '65e13c7687cc01de77ffaf4dae24cf874e7e522b4f432030c1b6e1e4b9ab13dc', 'api-key': 'HM_OVkhTCygs5BcVk2NY6aIi'}
{'symbol': 'SOLUSD', 'count': 1000, 'start': 2000, 'binSize': '1h', 'columns': 'close,timestamp'}
First row {'timestamp': '2022-01-11T12:00:00.000Z',

,domain,avg_price_usd,nb_sales,timestamp,txHash,fillSource,orderKind,price_sol,price_usd
0,000,320562.450900,1,1656872054,0x659f6961fb48db842f061250f6ac28e858e11b211d46...,opensea.io,wyvern-v2.3,2000.0,229240.0
1,porno,210410.170620,1,1656509031,0x64e33539108a54f905d0a082575bdc197a34bbdd59a1...,opensea.io,wyvern-v2.3,NaN,NaN
2,03jul,260989.689056,7,1653086759,0x8f0bf710aee447fb089187a4774c3cba4743f5ed032b...,x2y2.io,x2y2,NaN,NaN
3,sapdeutschland,179939.419940,1,1673820875,0xd71af68474266518c216f7405623648d7eb6604e1cf1...,opensea.io,seaport,NaN,NaN
4,ensmaxisnft,160720.573960,1,1658936856,0x7ff93b57a6c774aa6b3f5c3d640b7eb06c67b23a4b5b...,x2y2.io,x2y2,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7019,7046,1105.144370,1,1656154960,0x2b73fc861f0efccbbfa63e7eadbf212052ad57142d0e...,opensea.io,seaport,NaN,NaN
7020,5874,1285.213250,1,1678633823,0x96b78dcc14c844efd5b0097b3a774bb89ecdcdf35a67...,opensea.io,seaport-v1.4,NaN,NaN
7021,5473,1285.213250,1,1678633823,0x96b78dcc14c844efd5b0097b3a774bb89ecdcdf35a67...,opensea.io,seaport-v1.4,NaN,NaN
7022,8990,1613.291120,1,1654792951,0x646be57ba00d0d0c3fa28be09325a13b9e54d765f94d...,opensea.io,wyvern-v2.3,NaN,NaN


## Get registrability of domains on Bonfida
We only try to get registrability for ENS domains sold between 10k and 1k.
Which is a bad idea as we don't really reduce the search domain but only the opportunities above 10k

In [102]:
# All ENS sales matched on its unlisted counterpart on MagicEden (could be unlisted or available in the registry)
df_unlisted_sns_domains = df_sales[df_sales['price_sol'].isna()]
df_unlisted_sns_domains

,domain,avg_price_usd,nb_sales,timestamp,txHash,fillSource,orderKind,price_sol,price_usd
0,000,320562.450900,1,1656872054,0x659f6961fb48db842f061250f6ac28e858e11b211d46...,opensea.io,wyvern-v2.3,2000.0,220380.00
27,010,71585.463400,1,1668196679,0xcca57b1decf340424230c9ba1ca96fd444943fa9de08...,x2y2.io,x2y2,149.0,16418.31
51,meme,80450.510540,1,1688673323,0xc4447efea74c79afbd571cb0ca47fd47f3f81873e014...,opensea.io,seaport-v1.5,30000.0,3305700.00
79,653,23499.248310,3,1663848359,0x9baff7f29c90eca02c2ad996cfa32010e49998a66047...,ens.vision,seaport,95.0,10468.05
82,917,50384.351105,2,1663077111,0x5c89ca1c2b31d8270b5f9e5aeae044e4ad0a3de864d2...,ens.vision,seaport,5000.0,550950.00
...,...,...,...,...,...,...,...,...,...
6973,talents,1206.031290,1,1664087591,0x3dd1a3679a9b29a4124c1118c19983a50ba11bb0668a...,ens.domains,mint,10.0,1101.90
6975,louboutin,1660.494310,1,1655268880,0xfdb9256b109caffef7f3e8718fabe096ba058d54d89d...,ens.domains,mint,9.0,991.71
6976,fil,1569.884070,1,1696265495,0x387214320ffc35d644ca1a04ec4ae80ae61a13b7b075...,ens.domains,mint,100.0,11019.00
7001,dubs,1467.760370,1,1674511859,0xd798b1d971908460427bae19a6cb1290d2a3923284fe...,ens.vision,seaport,29.0,3195.51


In [ ]:
# Check unregistered SNS for ENS in between 10k and 1k
df_filtered_sales = df_unlisted_sns_domains[(df_unlisted_sns_domains['avg_price_usd']>1000)]

In [ ]:
# Check registration for SNS

# List of domains to check
domains = df_filtered_sales['domain'].tolist()

# Create a ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=3) as executor:
    # Submit tasks to the executor
    futures = [executor.submit(process_domain, domain) for domain in domains]

    # Create an empty dictionary to store results
    domain_status = {}

    # Retrieve results as they are completed
    for future in as_completed(futures):
        domain, is_registered = future.result()
        domain_status[domain] = is_registered

In [ ]:
# Convert the results dictionary back to a DataFrame
df_sns_to_be_registered = pd.DataFrame(list(domain_status.items()), columns=['domain', 'is_registered'])

# Filter not listed domains
df_sns_to_be_registered

In [ ]:
# Merge sns registered and sales
df_sales['is_registered']=df_sales.merge(df_sns_to_be_registered, how='left', on='domain')
df_sales['is_registered']=df_sales.apply(lambda x: x['is_registered']==True, axis=1)

# We suppose that unregistered domains cost 20$ (which is not rly true for rare domains etc..)
df_sales['price_usd']=df_sales.apply(lambda x:x['price_usd'] if x['is_registered'] else 20, axis=1)
df_sales

## Data management
### Dump in Pickle

In [ ]:
with open('df_sales.pkl', 'wb') as f:
    pickle.dump(df_sales, f)

### Load from disk

In [ ]:
with open('df_sales.pkl','rb') as file:
    df_sales=pickle.load(file)

## Data analysis
### Compute delta of SNS listed and avg sale price of ENS

In [108]:
# Filter available domains with the max delta between ENS and SNS
df_sales['delta'] = df_sales.apply(lambda x: x['avg_price_usd']-x['price_usd'])
df_sales[df_sales['is_registered']].sort_values(by=['delta','price_usd'], ascending=[False,True], inplace=True)

,domain,avg_price_usd,nb_sales,timestamp,txHash,fillSource,orderKind,price_sol,price_usd,delta
0,000,320562.450900,1,1656872054,0x659f6961fb48db842f061250f6ac28e858e11b211d46...,opensea.io,wyvern-v2.3,2000.0,2.203800e+05,1.001825e+05
27,010,71585.463400,1,1668196679,0xcca57b1decf340424230c9ba1ca96fd444943fa9de08...,x2y2.io,x2y2,149.0,1.641831e+04,5.516715e+04
183,576,45397.557070,1,1661434473,0xb2c5f94f7ec954da6935dcc5bbaacbabbd7c85fcf7a8...,opensea.io,seaport,57.6,6.346944e+03,3.905061e+04
356,354,46541.527980,1,1651830550,0x299b6e75a896ca8dc59ab44043796693c854530ac525...,opensea.io,wyvern-v2.3,99.0,1.090881e+04,3.563272e+04
199,npc,42799.745075,2,1702243319,0x9d6d83a6db0ec688910cb34608e56564d5ee067d1376...,x2y2.io,x2y2,69.0,7.603110e+03,3.519664e+04
...,...,...,...,...,...,...,...,...,...,...
6293,wix,1992.747440,1,1685992943,0x09817e00ffe938cc5e8280c8a37b9930e0422e58f0eb...,ens.domains,mint,5000.0,5.509500e+05,-5.489573e+05
4286,5608,2228.033470,2,1656430764,0x0856deb4d7f6314bc74802db2e42b8d510a403130ae9...,opensea.io,wyvern-v2.3,5608.0,6.179455e+05,-6.157175e+05
3872,gfy,2051.167860,1,1671369647,0x8aaf0f350182dec086dd3186110f1442d1d207965b43...,ens.domains,mint,15000.0,1.652850e+06,-1.650799e+06
51,meme,80450.510540,1,1688673323,0xc4447efea74c79afbd571cb0ca47fd47f3f81873e014...,opensea.io,seaport-v1.5,30000.0,3.305700e+06,-3.225249e+06


### Compute delta of SNS unregistered and avg sale price of ENS, CAREFUL SNS registration price varies from the assumed 20$

In [ ]:
# If not registered, we assume it's a 20$ domain
df_sales[not df_sales['is_registered']].sort_values(by=['delta','price_usd'], ascending=[False,True], inplace=True)

In [ ]:
###### !!! Doing a temporary merge for plane data analysis !!! ######